# Explore here

In [450]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [451]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from imblearn.metrics import specificity_score


In [452]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv')
pd.set_option('display.max_columns', None)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


Boosting para Clasificación

In [453]:
from sklearn.model_selection import train_test_split

X = df.drop("Outcome", axis = 1)
y = df["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [454]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators = 200, random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

PREDICCIÓN DEL MODELO

In [455]:
y_pred = model.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [456]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7077922077922078

In [457]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [458]:
# Evaluar el modelo  en entrenamiento y prueba
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [459]:
# Métricas del modelo complejo
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
Test,0.707792,0.615385,0.69596,0.580645,0.654545,0.737374
Diferencia,0.292208,0.384615,0.30404,0.419355,0.345455,0.262626


In [460]:
model_2 = XGBClassifier(n_estimators = 100, learning_rate = 0.01, gamma=5 ,max_depth=None,random_state = 42)
model_2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=5, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

PREDICCION DEL MODELO_2

In [461]:
y_pred= model_2.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [462]:
accuracy_score(y_test, y_pred)

0.7922077922077922

In [463]:
# Evaluar el modelo_2  en entrenamiento y prueba
train_pred_op = model_2.predict(X_train)
test_pred_op = model_2.predict(X_test)

In [464]:
# Métricas del modelo_2
get_metrics(y_train, y_test, train_pred_op, test_pred_op)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.807818,0.666667,0.748317,0.836879,0.553991,0.942643
Test,0.792208,0.680000,0.753535,0.755556,0.618182,0.888889
Diferencia,0.015610,-0.013333,-0.005218,0.081324,-0.064191,0.053755


implementación propia de la técnica de Boosting

In [473]:
from sklearn.tree import DecisionTreeClassifier


class CustomBoosting:

    def __init__(self, n_estimators=100, learning_rate=0.01, max_depth=10, min_samples_leaf=10, max_features=1.0, random_state=42):
        # Inicializa el modelo con hiperparámetros personalizables
        self.n_estimators = n_estimators  # Número de estimadores en el ensamble
        self.learning_rate = learning_rate  # Tasa de aprendizaje
        self.max_depth = max_depth  # Profundidad máxima de los árboles de decisión
        self.min_samples_leaf = min_samples_leaf  # Número mínimo de muestras por hoja
        self.max_features = max_features  # Proporción máxima de características a considerar en cada división
        self.random_state = random_state  # Semilla aleatoria para reproducibilidad
        self.estimadores = [] # Almacena los estimadores del ensamble

    def muestreo(self, X, y, pesos):
      # Realiza un muestreo ponderado de las instancias de entrenamiento
      np.random.seed(self.random_state)
      indices = np.random.choice(len(X), len(X), p=pesos)  # Selecciona posiciones del conjunto de datos con reemplazo
      return X[indices], y[indices]

    def get_error(self, y, y_pred, pesos):
      # Calcula el error ponderado, indicando cuántas instancias se clasificaron incorrectamente en relación a sus pesos
      instancias_erroneas = (y_pred != y)  # Identifica las instancias mal clasificadas (True/False)
      pesos_instancias_erroneas = pesos[instancias_erroneas]  # Obtiene los pesos de las instancias erróneas
      suma_peso_error = np.sum(pesos_instancias_erroneas)  # Suma los pesos de las instancias erróneas
      error_ponderado = suma_peso_error / np.sum(pesos)  # Proporciona un valor entre 0 y 1
      return instancias_erroneas, error_ponderado

    def get_estimator_importance(self, error_ponderado):
      # Calcula la importancia del estimador en el ensamble
      ratio_acierto_error = (1 - error_ponderado) / error_ponderado  # Mide el grado de acierto del estimador (mayor valor indica más aciertos que fallos)
      ratio_log = np.log(ratio_acierto_error)  # Aumenta el impacto de los estimadores que funcionan mejor y reduce el de los que funcionan peor
      estimador_imp = self.learning_rate * ratio_log  # Limita el impacto de cada estimador en la predicción final (para evitar el overfitting)
      return estimador_imp

    def update_weights(self, estimador_imp, instancias_erroneas, pesos):
      # Actualiza los pesos de las instancias para dar más importancia a las instancias mal clasificadas
      actualizacion_pesos = estimador_imp * instancias_erroneas  # Combina la importancia del estimador con las instancias mal clasificadas
      actualizacion_pesos = np.exp(actualizacion_pesos)  # Amplifica la influencia de las instancias erróneas en la próxima iteración
      pesos = pesos * actualizacion_pesos  # Aplica la actualización a los pesos
      pesos = pesos / np.sum(pesos)  # Normaliza los pesos para que sumen 1, creando una distribución de probabilidad válida
      return pesos

    def fit(self, X, y):
      # Inicializa los pesos de las instancias, todas con el mismo peso inicial
      pesos = np.ones(len(X)) / len(X)
      # Realiza el entrenamiento para el número de estimadores especificado
      for _ in range(self.n_estimators):
          # Realiza un muestreo aleatorio de las instancias de entrenamiento
          X_sampled, y_sampled = self.muestreo(X, y, pesos)
          # Ajusta un estimador (árbol de decisión) con las instancias muestreadas
          estimador = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X_sampled, y_sampled)
          # Realiza predicciones con el estimador
          y_pred = estimador.predict(X)
          # Calcula la importancia del error cometido por el estimador
          instancias_erroneas, error_ponderado = self.get_error(y, y_pred, pesos)
          # Calcula la importancia del modelo en el ensamble
          estimador_imp = self.get_estimator_importance(error_ponderado)
          # Almacena el estimador y su importancia asociada
          self.estimadores.append((estimador, estimador_imp))
          # Actualiza los pesos de las instancias
          pesos = self.update_weights(estimador_imp, instancias_erroneas, pesos)

    def predict(self, X):
      # Inicializa un vector de predicciones vacío con el mismo número de instancias a predecir
      final_predictions = np.zeros(X.shape[0])
      # Para cada estimador en el ensamble
      for estimador, estimador_imp in self.estimadores:
          # Realiza predicciones con las instancias
          predictions = estimador.predict(X)
          # Pondera las predicciones con la importancia del estimador
          preds_ponderadas = estimador_imp * predictions
          # Acumula las predicciones ponderadas de cada estimador
          final_predictions = final_predictions + preds_ponderadas
      # Realiza predicciones binarias (1 si es mayor que 0, 0 en caso contrario)
      yhats = np.where(final_predictions > 0, 1, 0)
      return yhats

In [466]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [474]:
# Entrenamos el modelo de Boosting
clf = CustomBoosting(n_estimators=100, learning_rate=0.01, max_depth=7, min_samples_leaf=7, max_features=X_train.shape[1]//2)
clf.fit(X_train.values, y_train.values)

In [476]:
# Realizamos predicciones en el conjunto de entrenamiento y prueba
test_pred_imp = clf.predict(X_test.values)
train_pred_imp = clf.predict(X_train.values)


In [477]:

# Evaluamos el rendimiento del modelo
get_metrics(y_train, y_test, train_pred_imp, test_pred_imp)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.500000,0.581173,0.617207,0.409615,1.0,0.234414
Test,0.461039,0.569948,0.580808,0.398551,1.0,0.161616
Diferencia,0.038961,0.011225,0.036399,0.011065,0.0,0.072798


Boosting para Regresión

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/clean_petrol_consumption_train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/clean_petrol_consumption_test.csv")

train_data.head()

,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,8.0,4447,8577,0.529,464
1,7.5,4870,2351,0.529,414
2,8.0,5319,11868,0.451,344
3,7.0,4345,3905,0.672,968
4,7.5,3357,4121,0.547,628


In [ ]:
X_train = train_data.drop(["Petrol_Consumption"], axis = 1)
y_train = train_data["Petrol_Consumption"]
X_test = test_data.drop(["Petrol_Consumption"], axis = 1)
y_test = test_data["Petrol_Consumption"]

In [ ]:
from xgboost import XGBRegressor

model_r = XGBRegressor(random_state = 42)
model_r.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

PREDICCIÓN DEL MODELO

In [ ]:
y_pred_r = model_r.predict(X_test)
y_pred_r

array([642.72314, 583.89545, 604.52313, 655.17633, 507.17087, 598.1727 ,
       508.003  , 922.8301 , 569.5492 , 586.8937 ], dtype=float32)

Para calcular la efectividad del modelo utilizaremos el error cuadrático medio (MSE):

In [ ]:
from sklearn.metrics import mean_squared_error

print(f"Error cuadrático medio: {mean_squared_error(y_test, y_pred_r)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_r)}")

Error cuadrático medio: 12462.193910804484
Coeficiente de determinación: -0.8575642037363085


Optimizado

In [ ]:
model_r2 = XGBRegressor(n_estimators = 100, learning_rate = 0.01, random_state = 42)
model_r2.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_r2 = model_r2.predict(X_test)
y_pred_r2

array([609.3015 , 597.8919 , 588.487  , 617.7691 , 539.5742 , 595.94116,
       554.8839 , 737.2859 , 580.9036 , 572.311  ], dtype=float32)

In [ ]:
print(f"Error cuadrático medio: {mean_squared_error(y_test, y_pred_r2)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_r2)}")

Error cuadrático medio: 5905.223264289275
Coeficiente de determinación: 0.11979131208154048
